In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW

2024-06-17 17:27:51.426817: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 17:27:51.426932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 17:27:51.565846: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
max_length=20
num_meter_classes = 9
batch_size = 128
learning_rate = 3e-5
num_epochs = 10
top = 1000000
take_top = False
def get_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [7]:
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('asafaya/bert-base-arabic')
model = BertForSequenceClassification.from_pretrained('asafaya/bert-base-arabic', num_labels=num_meter_classes)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-base-arabic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
meters = pd.read_csv('/kaggle/input/meters/meter_filtered.csv',encoding='utf-8-sig')
n = len(meters)
if(take_top):
  n = top
meters = meters[:n]
meters.head()

,الشطر,البحر
0,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا,الطويل
1,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ,الطويل
2,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً,الطويل
3,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ,الطويل
4,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ,الطويل


In [13]:
class ArabicPoetryDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
print(meters['البحر'].value_counts())

البحر
الطويل      263545
الكامل      222549
البسيط      147684
الخفيف       96540
الوافر       87412
الرجز        58262
المتقارب     42992
الرمل        42777
السريع       38239
Name: count, dtype: int64


In [12]:
meters['البحر'].unique()
label_dict = {label: i for i, label in enumerate(meters['البحر'].unique())}
label_dict

{'الطويل': 0,
 'المتقارب': 1,
 'الخفيف': 2,
 'الكامل': 3,
 'السريع': 4,
 'الوافر': 5,
 'البسيط': 6,
 'الرجز': 7,
 'الرمل': 8}

In [14]:
text = meters['الشطر']
label = meters['البحر']
label = label.apply(lambda x: label_dict[x])
texts = text.to_list()
labels = label.to_list()

In [15]:
dataset = ArabicPoetryDataset(texts, labels)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [18]:
print(f"number of batches : {len(dataloader)}")

number of batches : 977


In [19]:
from torch import nn
counts = [51720, 440698, 295574, 191568, 178820, 115762, 85472, 84778, 76166]
total_samples = sum(counts)
class_weights = [total_samples / count for count in counts]
weights_tensor = torch.tensor(class_weights, dtype=torch.float)
weights_tensor = weights_tensor.to(get_device())
loss_function = nn.CrossEntropyLoss(weight=weights_tensor)


In [21]:
def save_checkpoint(model, optimizer, epoch, iteration, filename="latest_checkpoint.pth"):
    checkpoint = {
        'epoch': epoch,
        'iteration': iteration,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }
    torch.save(checkpoint, filename)

In [33]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
model.train()
model.to(get_device())
losses = torch.tensor([])
for epoch in range(num_epochs):  # number of epochs
    i = 0
    print(f"number of batches : {len(dataloader)}")
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        input_ids = input_ids.to(get_device())
        attention_mask = attention_mask.to(get_device())
        labels = labels.to(get_device())
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits  # Assuming 'logits' is the attribute containing the tensor you need
        loss = loss_function(logits, labels)
        losses = losses.to(get_device())
        losses = torch.cat((losses, loss.view(1)))
        loss.backward()
        optimizer.step()
        print(f"iteration {i+1}, Loss: {loss.item()}")
        i += 1
        if (i) % 100 == 0:
                save_checkpoint(model, optimizer, epoch, i + 1)
    print(f"Epoch {epoch+1}, Loss: {loss.mean()}")

number of batches : 977
iteration 1, Loss: 0.1251075565814972
iteration 2, Loss: 0.14220422506332397
iteration 3, Loss: 0.12372241914272308
iteration 4, Loss: 0.13051819801330566
iteration 5, Loss: 0.16004164516925812
iteration 6, Loss: 0.08685296028852463
iteration 7, Loss: 0.14573946595191956
iteration 8, Loss: 0.1484624743461609


KeyboardInterrupt: 

In [ ]:
print(type(model))

In [34]:
def predict_meter(text, model, tokenizer):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    # Predict the meter using the model
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1)

    # Return the predicted meter class (as integer)
    return predictions.item()

In [37]:
number_of_correct_predictions = 0
num = 10000
for i in range(num):
  text_example = meters.iloc[i]
  #print(text_example['البحر'])
  predicted_meter = predict_meter(text_example['الشطر'], model, tokenizer)
  labelling = {v: k for k, v in label_dict.items()}
  #print(f"Predicted Meter Class: {labelling[predicted_meter]}")
  if(labelling[predicted_meter] == text_example['البحر']):
    number_of_correct_predictions += 1
print(f"Number of correct predictions: {(number_of_correct_predictions/num)*100}")


Number of correct predictions: 98.52


In [45]:
number_of_correct_predictions = 0
num = 100
random_integers = torch.randint(0, 1000000, (num,))
for i in random_integers:
  text_example = meters.iloc[i.item()]
  predicted_meter = predict_meter(text_example['الشطر'], model, tokenizer)
  labelling = {v: k for k, v in label_dict.items()}
  print(f"text : {text_example['الشطر']}, meter : {text_example['البحر']}, Predicted Meter Class: {labelling[predicted_meter]}")
  if(labelling[predicted_meter] == text_example['البحر']):
    number_of_correct_predictions += 1
print(f"Number of correct predictions: {(number_of_correct_predictions/num)*100}")


text : ماذا أَقول وَفي فَمي, meter : الكامل, Predicted Meter Class: الكامل
text : فَمَتى عَطَستُ فإنّ أنفِيَ أجدعٌ, meter : الكامل, Predicted Meter Class: الكامل
text : من الجانبِ الشرقيّ نُوديَ كلُّ مَنْ, meter : الطويل, Predicted Meter Class: الطويل
text : لَيسَت بِشَوشاةِ الحَديثِ وَلا, meter : الكامل, Predicted Meter Class: الكامل
text : كَأَنَّ المِسكَ دُقَّ لَها فَصيغَت, meter : الوافر, Predicted Meter Class: الوافر
text : وَتَرَكتُ حُلوَ العَيشِ لَم أَحفِل بِهِ, meter : الكامل, Predicted Meter Class: الكامل
text : بالله جد على الطلا, meter : الكامل, Predicted Meter Class: الكامل
text : ذو حجرٍ نسبةً وغيظاً, meter : البسيط, Predicted Meter Class: البسيط
text : أمِطْ عنْكَ مهْما اسْطَعْتَ كلَّ إرادَةٍ, meter : الطويل, Predicted Meter Class: الطويل
text : هم العلماء إن ذكرت علوم, meter : الوافر, Predicted Meter Class: الوافر
text : قُلتُ نَعَم قالَ وَفي مُطرِبٍ, meter : السريع, Predicted Meter Class: السريع
text : هل ظنّ ثَغْرَكِ أُقحُواناً ناضراً, meter : الكامل, Predicted Meter C